# SPAA - Simple Py Audio Analysis

## introduction:
the aim of SPAA it to preform simple analysis on audio files it is strongly inspired by this article from KD Nuggets: https://www.kdnuggets.com/2020/02/audio-data-analysis-deep-learning-python-part-1.html



### Definition of the environement Pre requisites 
Execute the code of the following cell only once after you can coment it out and set your vscode or Jupiter to work in the SPAA environement

### Instalation of pre requisites:
In a terminal or a shell execute the following commands:
> conda create --name SPAA_env python=3.10 numpy scipy matplotlib ipywidgets ipykernel -y

> conda activate SPAA_env

> conda install librosa scikit-learn -y



In [ ]:
import librosa
import sklearn
import librosa.display

import numpy as np
import matplotlib.pyplot as plt
import IPython.display   as ipd

print(f"numpy   : {np.__version__ }")
print(f"librosa : {librosa.__version__ }")
print(f"sklearn : {sklearn.__version__ }")


### The file you want to analyse

Folowing are a few public domain files exemples to test the analysis you can of course use your own Wave or MP3

In [ ]:
#wave_file = "bells-tibetan-daniel_simon.wav"    # https://soundbible.com/2205-Bells-Tibetan-Large.html
#wave_file = "Large SUV Pass By-SoundBible.com-1354956481.wav" # https://soundbible.com/611-Large-SUV-Pass-By.html
#wave_file = "airplane-takeoff_daniel_simion.wav" # https://soundbible.com/2195-Airplane-Takeoff.html
#wave_file = "human-heartbeat-daniel_simon.wav"  # https://soundbible.com/2162-Human-Heartbeat.html 


### Listen to the file

In [ ]:
ipd.Audio(wave_file)

### Basic information about the audio data

In [ ]:
x , sr         = librosa.load(wave_file, sr=None)
n_samples      = x.shape[0]
zero_crossings = librosa.zero_crossings(x, pad=False)
n_zero_cross   = sum(zero_crossings)
smpl_per_z_x   = n_samples / n_zero_cross
num_zeroes     = n_samples - len(x.nonzero()[0])
abs_max        = max(abs(x.max()), abs(x.min()))
print(type(x), type(sr))#<class 'numpy.ndarray'> <class 'int'>print(x.shape, sr)#(94316,) 22050
print(f"shape       : {x.shape}")
print(f"sample rate : {sr}")
print(f"Size        : {x.nbytes/1000000} Mb")
print(f"Max Value   : {x.max()}")
print(f"Min Value   :{x.min()}")
print(f"Max abs Val : {abs_max}")
print(f"Number  of  Zeroes crossing: {n_zero_cross} z_Xings")
print(f"Samples per Zeroes crossing: {smpl_per_z_x:8.2f} smpl/z_Xings")
print(f"Zeroes crossing rate       : {1/(smpl_per_z_x/sr):7.1f} z_Xings/s")
print(f"Zeroes      : {num_zeroes}")
print(f"Zeroes      : {100 * num_zeroes / n_samples:6.2f}%")
print(f"Mean        : {100 * x.mean()   / (abs_max) :7.3f}%")
#x.cumsum


In [ ]:
p_start    = 0.4  # fraction of the streal where the sample window starts
window_siz = 1000  # the number of sample to display
n0 = int(p_start * n_samples)
n1 = n0 + window_siz
plt.figure(figsize=(15, 6))
librosa.display.waveshow(x, sr=sr)
# Zooming in
#plt.figure(figsize=(15, 6))
#plt.plot(x[n0:n1])
#plt.grid()

In [ ]:
t_start_s =      11.0 * 60
t_end_s   =      17.0 * 60
s_start   = int(t_start_s * sr)
s_end     = int(  t_end_s * sr)
x_cut     = x[s_start:s_end]

n_samples      = x_cut.shape[0]
zero_crossings = librosa.zero_crossings(x_cut, pad=False)
n_zero_cross   = sum(zero_crossings)
smpl_per_z_x   = n_samples / n_zero_cross
num_zeroes     = n_samples - len(x_cut.nonzero()[0])
abs_max        = max(abs(x_cut.max()), abs(x_cut.min()))
print(type(x), type(sr))#<class 'numpy.ndarray'> <class 'int'>print(x.shape, sr)#(94316,) 22050
print(f"shape       : {x_cut.shape}")
print(f"sample rate : {sr}")
print(f"Size        : {x_cut.nbytes/1000000} Mb")
print(f"Max Value   : {x_cut.max()}")
print(f"Min Value   :{x_cut.min()}")
print(f"Max abs Val : {abs_max}")
print(f"Number  of  Zeroes crossing: {n_zero_cross} z_Xings")
print(f"Samples per Zeroes crossing: {smpl_per_z_x:8.2f} smpl/z_Xings")
print(f"Zeroes crossing rate       : {1/(smpl_per_z_x/sr):7.1f} z_Xings/s")
print(f"Zeroes      : {num_zeroes}")
print(f"Zeroes      : {100 * num_zeroes / n_samples:6.2f}%")
print(f"Mean        : {100 * x_cut.mean()   / (abs_max) :7.3f}%")
#x.cumsum

In [ ]:
p_start    = 0.4  # fraction of the streal where the sample window starts
window_siz = 1000  # the number of sample to display
n0 = int(p_start * n_samples)
n1 = n0 + window_siz
plt.figure(figsize=(15, 6))
librosa.display.waveshow(x_cut, sr=sr)

In [ ]:
n_fft = 8192
plt.figure(figsize=(16, 6))
ft = np.abs(librosa.stft(x_cut[:n_fft], hop_length = n_fft+1))
plt.plot(ft);
plt.title('Spectrum');
plt.xlabel('Frequency Bin');
plt.ylabel('Amplitude');

In [ ]:
x_fft      = librosa.stft(x_cut, n_fft=n_fft, win_length=n_fft)
x_fft_db   = librosa.amplitude_to_db(abs(x_fft))
x_fft_db_2 = np.square(x_fft_db) 
print(type(x_fft_db  ))
print(x_fft_db.shape  )
print(type(x_fft_db_2.shape))
print(x_fft_db_2.shape  )

abs_max        = max(abs(x_fft_db_2.max()), abs(x_fft_db_2.min()))
print(type(x), type(sr))#<class 'numpy.ndarray'> <class 'int'>print(x.shape, sr)#(94316,) 22050
print(f"shape       : {x_fft_db_2.shape}")
print(f"Size        : {x_fft_db_2.nbytes/1000000} Mb")
print(f"Max Value   : {x_fft_db_2.max()}")
print(f"Min Value   :{x_fft_db_2.min()}")
print(f"Max abs Val : {abs_max}")
print(f"Mean        : {100 * x_fft_db_2.mean()   / (abs_max) :7.3f}%")
#x.cumsum

In [ ]:
plt.figure(figsize=(16, 6))
plt.plot(x_fft_db);
plt.title('Spectrum');
plt.xlabel('Frequency Bin');
plt.ylabel('Amplitude');

In [ ]:
n_fft = 2048
S = librosa.amplitude_to_db(abs(x_cut))
plt.figure(figsize=(16, 4))
librosa.display.specshow(S, sr=sr, hop_length=512, x_axis='time', y_axis='linear')
plt.colorbar(format='%+2.0f dB')
plt.show()

In [ ]:
#x_fft    = librosa.stft(x_cut, n_fft=1024, win_length=1024)
#x_fft_db = librosa.amplitude_to_db(abs(x_fft))

fig, ax = plt.subplots(figsize=(16, 6))  
img     = librosa.display.specshow(x_fft_db_2, sr=sr, x_axis='time', y_axis='hz')
ax.set_title('Spectogram intensity', fontsize=14)
ax.set_ylim(0,30000 )
plt.colorbar(img, ax=ax, format=f'%0.2f')

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))  
img     = librosa.display.specshow(x_fft_db, sr=sr, x_axis='time', y_axis='log', ax=ax)
ax.set_title('Spectogram Db', fontsize=14)
ax.set_ylim(0,26000 )
plt.colorbar(img, ax=ax, format=f'%0.2f')

In [ ]:
mel     = librosa.feature.melspectrogram(y=x, sr=sr, n_mels=256)
mel_db  = librosa.amplitude_to_db(mel, ref=np.max)

fig, ax = plt.subplots(figsize=(15, 6))  
img     = librosa.display.specshow(mel_db, x_axis='time', y_axis='log', ax=ax)
ax.set_title('Mel Spectogram', fontsize=14)
fig.colorbar(img, ax=ax, format=f'%0.2f')
plt.show()

In [ ]:
#x_fmt    = librosa.fmt(x, t_min=0.5, n_fmt=None, kind='cubic', beta=0.5, over_sample=1, axis=-1)
#x_fmt_db = librosa.amplitude_to_db(abs(x_fmt))
x_hcqt    = librosa.hybrid_cqt(x)
x_hcqt_db = librosa.amplitude_to_db(abs(x_hcqt))
fig, ax = plt.subplots(figsize=(15, 6))  
img     = librosa.display.specshow(x_hcqt_db, sr=sr, x_axis='time', y_axis='hz')
ax.set_title('Compute the hybrid constant-Q transform', fontsize=14)
#ax.set_ylim(0,13000 )
plt.colorbar(img, ax=ax, format=f'%0.2f')

In [ ]:
x_iirt    = librosa.iirt(x)
x_iirt_db = librosa.amplitude_to_db(abs(x_iirt))
fig, ax = plt.subplots(figsize=(15, 6))  
img     = librosa.display.specshow(x_iirt_db, sr=sr, x_axis='time', y_axis='hz')
ax.set_title('Time-frequency representation using IIR filters', fontsize=14)
#ax.set_ylim(0,13000 )
plt.colorbar(img, ax=ax, format=f'%0.2f')

In [ ]:
hop_length = 256

chromagram = librosa.feature.chroma_stft(x, sr=sr, hop_length=hop_length)
fig, ax    = plt.subplots(figsize=(15, 6))  
img        = librosa.display.specshow(chromagram, x_axis='time', y_axis='chroma', hop_length=hop_length, cmap='coolwarm')
ax.set_title('Chroma SFT', fontsize=14)
#ax.set_ylim(0,13000 )
plt.colorbar(img, ax=ax, format=f'%0.2f')

In [ ]:
mfccs = librosa.feature.mfcc(x, sr=sr)
print(mfccs.shape)
#(20, 97)
#Displaying  the MFCCs:
fig, ax    = plt.subplots(figsize=(15, 6))  
img        = librosa.display.specshow(mfccs, sr=sr, x_axis='time')
ax.set_title('MFCC', fontsize=14)
#ax.set_ylim(0,13000 )
plt.colorbar(img, ax=ax, format=f'%0.2f')

In [ ]:
spectral_centroids = librosa.feature.spectral_centroid(x, sr=sr)[0]
spectral_centroids.shape
(775,)
# Computing the time variable for visualization
plt.figure(figsize=(12, 4))
frames = range(len(spectral_centroids))
t = librosa.frames_to_time(frames)
# Normalising the spectral centroid for visualisation
def normalize(x, axis=0):
    return sklearn.preprocessing.minmax_scale(x, axis=axis)
#Plotting the Spectral Centroid along the waveform
librosa.display.waveshow(x, sr=sr, alpha=0.4)
plt.plot(t, normalize(spectral_centroids), color='b')

In [ ]:


spectral_rolloff = librosa.feature.spectral_rolloff(x+0.01, sr=sr)[0]
plt.figure(figsize=(12, 4))
librosa.display.waveshow(x_cut, sr=sr, alpha=0.4)
plt.plot(t, normalize(spectral_rolloff), color='r')



In [ ]:

spectral_bandwidth_2 = librosa.feature.spectral_bandwidth(x+0.01, sr=sr)[0]
spectral_bandwidth_3 = librosa.feature.spectral_bandwidth(x+0.01, sr=sr, p=3)[0]
spectral_bandwidth_4 = librosa.feature.spectral_bandwidth(x+0.01, sr=sr, p=4)[0]
plt.figure(figsize=(15, 9))
librosa.display.waveshow(x_cut, sr=sr, alpha=0.4)
plt.plot(t, normalize(spectral_bandwidth_2), color='r')
plt.plot(t, normalize(spectral_bandwidth_3), color='g')
plt.plot(t, normalize(spectral_bandwidth_4), color='y')
plt.legend(('p = 2', 'p = 3', 'p = 4'))

